In [1]:
import numpy as np
import pandas as pd

# EDA
train = pd.read_csv("/kaggle/input/titanic/train.csv")
#train.head
train.info
# train.shape
# train.isnull().sum()
# train['Embarked'].value_counts()
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [2]:
train['Survived'].value_counts()

0    549
1    342
Name: Survived, dtype: int64

549 passengers died, and 342 passengers survived. Of those survivng passengers, what characteristics did they have that may have contributed to their survival? 

In [3]:
train['Sex'].value_counts(normalize=True)

male      0.647587
female    0.352413
Name: Sex, dtype: float64

64% of the passengers are male, and 35% of the passengers are female. Because there are more male passengers, it can be mistakenly concluded that being male came with a higher chance of surviving/not surviving. We wouldn't necessarily know if the same could be said for female passengers, as there isn't an equal amount of them and therefore their results to compare to males, and vice-versa.

In [4]:
my_agg=(train
 .groupby(['Sex','Survived'])
 .agg(
     fates=pd.NamedAgg('Survived', 'count')))
print(my_agg)

                 fates
Sex    Survived       
female 0            81
       1           233
male   0           468
       1           109


More females survived than died, and more males died than survived. What other factors might have played into this?

In [5]:
train['Age'].describe()

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

In [6]:
class Quantile:
    def __init__(self, q):
        self.q=q
    def __call__(self, x):
        return x.quantile(self.q)
    
print(
train
.groupby(['Sex', 'Survived', 'Age'])
.agg(
    b1=('Age', 'size'),
    )
)


                      b1
Sex    Survived Age     
female 0        2.0    4
                3.0    1
                6.0    1
                8.0    1
                9.0    4
...                   ..
male   1        52.0   1
                56.0   1
                60.0   1
                62.0   1
                80.0   1

[216 rows x 1 columns]


In [7]:
train.where(train.Age <= 20)\
    .groupby([train.Sex,train.Survived])\
    .agg(
        under21=pd.NamedAgg('Survived', 'count'))

under21
Sex    Survived         
female 0              24
       1              53
male   0              73
       1              29

In [8]:
train.groupby(['Sex', 
               'Survived', 
               pd.cut(x=train['Age'],
                      bins=train['Age'].quantile([0,.25,.5,.75,1]),
                      labels=['0-20','21-28','29-38','38-80'])
              ])\
      .size()\
      .unstack()

Age              0-20  21-28  29-38  38-80
Sex    Survived                           
female 0           24     16     10     14
       1           53     47     51     46
male   0           73    101     89     97
       1           28     19     25     20